In [28]:
model_gen_path_list=[]
for i in range(1,8):
        current_path='/gpfs/fs1/projects/gpu_adlr/datasets/dasu/prompting/predicted/TQA/357m/output_answer_generations_k10_357m_gc_multisetdpr_queryctx_p0.9_all_rnd{}.txt'.format(i)
        model_gen_path_list.append(current_path)

In [6]:
model_gen_path_list[0]

'/gpfs/fs1/projects/gpu_adlr/datasets/dasu/prompting/predicted/TQA/357m/output_answer_generations_k10_357m_gc_multisetdpr_queryctx_p0.9_all_rnd1.txt'

In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [29]:
answer_list_list=[]
from tqdm import tqdm
import regex
import string
import json

def normalize_answer(s):
    def remove_articles(text):
        return regex.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def read_answer_list(prediction_file_list):
    prediction_list_list = []
    for i, prediction_file in enumerate(prediction_file_list):
        prediction_list = []
        print('reading %s' % prediction_file)
        with open(prediction_file, "r") as f:
            for i, line in enumerate(tqdm(f)):
                line = line.replace("Answer:","")
                line = line.replace("Answer: ","")
                line = line.replace('????  ', "")
                line = line.replace('A: ',"")
                line = line.replace("A:", "")

                line = line.strip()

                if "<|endoftext|>" in line:
                    line = line.replace("<|endoftext|>", "")
                line = normalize_answer(line) # normalize the answer
                prediction_list.append(line)

        prediction_list_list.append(prediction_list)
    return prediction_list_list

def read_ground_truth(ground_truth_file):
    ground_truths_list = []
    
    if ground_truth_file.endswith(('txt', 'lst')):
        raw_data = open(ground_truth_file, 'r')
    else:
        with open(ground_truth_file, 'r') as f:
            raw_data = json.load(f)

    for each in raw_data:
        if ground_truth_file.endswith('txt'):
            each = json.loads(each)
        
        if 'answers' in each:
            ground_truths_list.append(each['answers'])
        elif 'answer' in each:
            ground_truths_list.append(each['answer'])
        else:
            ground_truths_list.append([each])
            
    return ground_truths_list

def ems(prediction, ground_truths):
    return max([exact_match_score(prediction, gt) for gt in ground_truths])

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

def cal_ems(ground_truths_list, prediction_list):
    exactmatch_list = []

    for i,each in enumerate(prediction_list):
        # print("=============")
        # print(each)
        # print(ground_truths_list[i])
        score = ems(each, ground_truths_list[i])
        # print(score)
        exactmatch_list.append(score)
    
    return exactmatch_list


In [30]:
answer_list_list = read_answer_list(model_gen_path_list)

reading /gpfs/fs1/projects/gpu_adlr/datasets/dasu/prompting/predicted/TQA/357m/output_answer_generations_k10_357m_gc_multisetdpr_queryctx_p0.9_all_rnd1.txt


11313it [00:00, 49212.82it/s]


reading /gpfs/fs1/projects/gpu_adlr/datasets/dasu/prompting/predicted/TQA/357m/output_answer_generations_k10_357m_gc_multisetdpr_queryctx_p0.9_all_rnd2.txt


11313it [00:00, 62963.82it/s]


reading /gpfs/fs1/projects/gpu_adlr/datasets/dasu/prompting/predicted/TQA/357m/output_answer_generations_k10_357m_gc_multisetdpr_queryctx_p0.9_all_rnd3.txt


...

11313it [00:00, 70441.59it/s]


reading /gpfs/fs1/projects/gpu_adlr/datasets/dasu/prompting/predicted/TQA/357m/output_answer_generations_k10_357m_gc_multisetdpr_queryctx_p0.9_all_rnd6.txt


11313it [00:00, 69278.80it/s]


reading /gpfs/fs1/projects/gpu_adlr/datasets/dasu/prompting/predicted/TQA/357m/output_answer_generations_k10_357m_gc_multisetdpr_queryctx_p0.9_all_rnd7.txt


11313it [00:00, 70872.76it/s]


In [8]:
ground_truth_file='/gpfs/fs1/projects/gpu_adlr/datasets/dasu/open_domain_data/TQA/test.json'
ground_truths_list = read_ground_truth(ground_truth_file)

In [31]:
answer_list_list[0][0]

'david seville'

In [32]:
from collections import Counter

def marginalize_prediction(prediction_list_list):
    
    most_predicted_answer_list = []
    l = len(prediction_list_list[0])
    for j in range(10):
        current_list=[]
        for i in range(len(prediction_list_list)):
            current_list.append(prediction_list_list[i][j])
        x = Counter(current_list)
        print("===="*10)
        print(x)
        (most_predicted_answer, _) = x.most_common()[0]
        index = current_list.index(most_predicted_answer)
        print(index)
        most_predicted_answer_list.append(most_predicted_answer)
    
    return most_predicted_answer_list

In [33]:
margin_answer_list = marginalize_prediction(answer_list_list)

Counter({'david seville': 7})
Counter({'taurus': 6, 'virgo': 1})
Counter({'music of night': 4, 'music of night stars as mariah carey and norah jones walt disney studios produced featuring guest star debbie harry twohour musical about ageing': 1, 'music of night all i ask of you music': 1, 'dear evan hansen dear evan hansen is musical by andrew lloyd webber production was premiered on 10': 1})
Counter({'marquess of salisbury': 3, 'sir henry campbellbannerman': 2, 'lord roberts althorp': 1, 'lord salisbury': 1})
Counter({'beatles': 7})
Counter({'frank sinatra': 4, 'mirror': 1, 'kathleen ferrier': 1, 'anne oldfield': 1})
Counter({'octopussy': 7})
Counter({'58': 2, '62': 1, '746 million': 1, '37 million': 1, '449 million': 1, '54 million': 1})
Counter({'california': 5, 'new jersey': 1, 'tennessee': 1})
Counter({'gloria swanson': 2, 'marilyn monroe marilyn monroe born marilyn monroe june 12 1925 –': 1, 'rita moreno rita moreno born rita louise': 1, 'marilyn monroe marilyn monroe born marily

In [ ]:
answer_list_list[0]

In [55]:
x=Counter({'kathleen ferrier': 13, 'frank sinatra': 9, 'mirror': 1, 'anne oldfield': 1, 'murder': 1, 'savoy ballroom fire': 1, 'derani oliver': 1, 'death': 1, 'linda mccartney': 1, 'sheilah hamilton': 1, 'her brother roger cooper': 1})

In [58]:
x.most_common()[1]

('frank sinatra', 9)

In [51]:
margin_answer_list[3]

'sir henry campbellbannerman'

In [50]:
ground_truths_list[3]

['Sir Henry Campbell-Bannerman',
 'Campbell-Bannerman',
 'Campbell Bannerman',
 'Sir Henry Campbell Bannerman',
 'Henry Campbell Bannerman',
 'Henry Campbell-Bannerman']

In [28]:
def or_prediction(prediction_list_list, ground_truths_list, l):
    exactmatch_list_list = []
    for prediction in prediction_list_list:
        exactmatch_list = cal_ems(ground_truths_list, prediction[:l])
        exactmatch_list_list.append(exactmatch_list)

    new_exactmatch_list=[]
    true_list_list = []
    # l=len(exactmatch_list_list[0])
    for j in range(l):
        sum = 0
        true_list=[]
        for i in range(len(exactmatch_list_list)):
            true_list.append(int(exactmatch_list_list[i][j]))
            sum += int(exactmatch_list_list[i][j])
        new_exactmatch_list.append(bool(sum))
        true_list_list.append(true_list)
    
    return new_exactmatch_list, true_list_list

In [37]:
new_exactmatch_list, true_list_list = or_prediction(answer_list_list, ground_truths_list[:10],10)

In [54]:
import numpy as np
new_score = np.mean(new_exactmatch_list)
new_score

0.0

In [52]:
true_list_list[3]


[0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1]

In [39]:
new_exactmatch_list

[True, False, False, True, False, False, True, False, False, False]

In [53]:
margin_result = cal_ems(ground_truths_list[:10], margin_answer_list)

In [54]:
margin_result

[True, False, False, True, False, False, True, False, False, False]